<a href="https://colab.research.google.com/github/PuchToTalk/Football_market-value/blob/Airflow-dag/my_dag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1ere version

In [ ]:
from datetime import datetime, timedelta

from airflow import DAG
from airflow.operators.python import PythonOperator
import pandas as pd



with DAG(
       'my_first_dag',
       default_args={
           'depends_on_past': False,
           'email': ['airflow@example.com'],
           'email_on_failure': False,
           'email_on_retry': False,
           'retries': 1,
           'retry_delay': timedelta(minutes=5),
       },
       description='A first DAG',
       schedule=None,
       start_date=datetime(2023, 1, 1),
       catchup=False,
       tags=['example'],

) as dag:
   dag.doc_md = """
       This is my DAG in airflow for the Big Data Project.
       I can write documentation in Markdown here with **bold text** or __bold text__.
   """

   import pandas as pd
   import pyarrow as pa
   import pyarrow.parquet as pq


   def load_foot_perf():
       print("load foot perf task")
       # Chargement du fichier foot_perf.csv
       foot_perf_data = pd.read_csv("/Users/paulc/airflow/dags/foot_perf.csv")
       #foot_perf_data = foot_perf_data.to_json(orient='records')

       # Convert DataFrame to PyArrow Table
       foot_perf_data = pa.Table.from_pandas(foot_perf_data)
       output_1 = 'output_foot_perf.parquet'
       pq.write_table(foot_perf_data, output_1)

       # Read Parquet file back into a DataFrame
       parquet_table1 = pq.read_table(output_1)
       foot_perf_data = parquet_table1.to_pandas()
       foot_perf_data = foot_perf_data.to_json(orient='records')
       return foot_perf_data

   foot_perf_data = load_foot_perf()
   #print(foot_perf_data)



   def load_market_value():
       print("load market value task")
       # Chargement du fichier market_value.csv
       market_value_data = pd.read_csv("/Users/paulc/airflow/dags/only_mv2.csv")
       #market_value_data = market_value_data.to_json(orient='records')
       # Convert DataFrame to PyArrow Table
       market_value_data = pa.Table.from_pandas(market_value_data)
       output_2 = 'output_foot_value.parquet'
       pq.write_table(market_value_data, output_2)

       # Read Parquet file back into a DataFrame
       parquet_table2 = pq.read_table(output_2)
       market_value_data = parquet_table2.to_pandas()
       market_value_data = market_value_data.to_json(orient='records')
       return market_value_data


   market_value_data = load_market_value()
   #print(market_value_data)


   def clean_foot_perf():
       print("cleaned foot perf data task")
       # Nettoyage et filtrage des données de foot_perf
       foot_perf_data = pd.read_json(load_foot_perf())
       # Read Parquet file back into a DataFrame
       output_1 = 'output_foot_perf.parquet'
       # Read Parquet file back into a DataFrame
       parquet_table1 = pq.read_table(output_1)
       foot_perf_data = parquet_table1.to_pandas()
       return foot_perf_data


   print(clean_foot_perf())


   # Perform further operations with cleaned_foot_perf_data if needed

   def clean_market_value(market_value_data):
       print("formatted market value data task")
       market_value_data = pd.DataFrame(market_value_data)
       filtered_data = market_value_data.drop(columns= filtered_data.columns[0], axis=1)
       filtered_data = filtered_data.drop(["Nation", "Pos"], axis=1)
       filtered_data = filtered_data.to_dict(orient='records')
       return filtered_data


   def join_datasets(df_football_stats, market_value_data):
       print("join 2 datasets task ")
       # Jointure des données de foot_perf et market_value
       merged_data = df_football_stats.merge(market_value_data, on='player', how='inner')
       return merged_data


   def index_to_elastic(merged_data):
       print("merged data task")
       # Indexation des données dans ElasticSearch
       # Code pour l'indexation des données dans ElasticSearch
       #pass

   source_to_raw1 = PythonOperator(
        task_id='source_to_raw1',
        python_callable=load_foot_perf,
        #provide_context=True,  # Add this line to pass the context to the function
        #op_kwargs={'foot_perf_data': load_foot_perf()}  # Initialize foot_perf_data with None
    )

   source_to_raw2 = PythonOperator(
       task_id='source_to_raw2',
       python_callable=load_market_value,
       #provide_context=True,  # Add this line to pass the context to the function
       #op_kwargs={'market_value_data': load_market_value()}  # Initialize foot_perf_data with None
   )

   raw_to_formatted1 = PythonOperator(
       task_id='raw_to_formatted1',
       python_callable=clean_foot_perf,
       #op_kwargs={'foot_perf_data': '{{ ti.xcom_pull(task_ids="source_to_raw1") }}'}

   )

   raw_to_formatted2 = PythonOperator(
        task_id='raw_to_formatted2',
        python_callable=clean_market_value,
        #op_kwargs={'market_value_data': '{{ ti.xcom_pull(task_ids="source_to_raw2") }}'}

    )

   produce_usage = PythonOperator(
       task_id='produce_usage',
       python_callable=join_datasets,

   )

   index_to_elastic = PythonOperator(
       task_id='index_to_elastic',
       python_callable=index_to_elastic,

   )

   source_to_raw1 >> raw_to_formatted1
   source_to_raw2 >> raw_to_formatted2
   [raw_to_formatted1, raw_to_formatted2] >> produce_usage >> index_to_elastic


   if __name__ == 'main':
        df = clean_foot_perf(load_foot_perf())
        df1 = clean_market_value(load_market_value())
        df2 = join_datasets(df, df1)
        index_to_elastic(df2)

2eme version : méthode JSON -> Data -> JSON

In [ ]:
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.models import DagRun
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

default_args = {
    'depends_on_past': False,
    'email': ['airflow@example.com'],
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

dag = DAG(
    'my_first_dag',
    default_args=default_args,
    description='A first DAG',
    schedule=None,
    start_date=datetime(2023, 1, 1),
    catchup=False,
    tags=['example'],
)

def load_foot_perf():
    print("load foot perf task")
    foot_perf_data = pd.read_csv("/Users/paulc/airflow/dags/foot_perf.csv")
    print(foot_perf_data)
    foot_perf_data_parquet = pa.Table.from_pandas(foot_perf_data)
    output_1 = 'output_foot_perf.parquet'
    pq.write_table(foot_perf_data_parquet, output_1)
    foot_perf_data = foot_perf_data.to_json(orient='records')
    print(foot_perf_data)
    return foot_perf_datax

def load_market_value():
    print("load market value task")
    market_value_data = pd.read_csv("/Users/paulc/airflow/dags/only_mv2.csv")
    market_value_data_parquet = pa.Table.from_pandas(market_value_data)
    output_2 = 'output_foot_value.parquet'
    pq.write_table(market_value_data_parquet, output_2)
    market_value_data = market_value_data.to_json(orient='records')
    return market_value_data

def clean_foot_perf(ti):
    print("cleaned foot perf data task")
    foot_perf_data = ti.xcom_pull(task_ids='source_to_raw1')
    # Perform cleaning operations on foot_perf_data DataFrame
    cleaned_df_perf = pd.read_json(foot_perf_data)
    print(cleaned_df_perf)
    cleaned_df_perf_data = cleaned_df_perf.to_json(orient='records')
    return cleaned_df_perf_data


def clean_market_value(ti):
    print("formatted market value data task")
    market_value_data = ti.xcom_pull(task_ids='source_to_raw2')
    # Perform cleaning operations on market_value_data DataFrame
    cleaned_df_value = pd.read_json(market_value_data)
    print(cleaned_df_value)
    cleaned_df_value_data = cleaned_df_value.to_json(orient='records')
    return cleaned_df_value_data


def join_datasets(ti):
    print("join 2 datasets task")
    foot_perf_data = ti.xcom_pull(task_ids='raw_to_formatted1')
    market_value_data = ti.xcom_pull(task_ids='raw_to_formatted2')
    # Join foot_perf_data and market_value_data
    merged_data = foot_perf_data.merge(market_value_data, on='player', how='inner')
    return merged_data

def index_to_elastic(ti):
    print("index to ElasticSearch task")
    merged_data = ti.xcom_pull(task_ids='combine')
    # Index merged_data to ElasticSearch
    # ...

source_to_raw1 = PythonOperator(
    task_id='source_to_raw1',
    python_callable=load_foot_perf,
    dag=dag,
)

source_to_raw2 = PythonOperator(
    task_id='source_to_raw2',
    python_callable=load_market_value,
    dag=dag,
)

raw_to_formatted1 = PythonOperator(
    task_id='raw_to_formatted1',
    python_callable=clean_foot_perf,

    dag=dag,
)

raw_to_formatted2 = PythonOperator(
    task_id='raw_to_formatted2',
    python_callable=clean_market_value,

    dag=dag,
)

combine = PythonOperator(
    task_id='combine',
    python_callable=join_datasets,

    dag=dag,
)

index_to_elastic = PythonOperator(
    task_id='index_to_elastic',
    python_callable=index_to_elastic,

    dag=dag,
)

source_to_raw1 >> raw_to_formatted1
source_to_raw2 >> raw_to_formatted2
[raw_to_formatted1, raw_to_formatted2] >> combine >> index_to_elastic
